# LangGraph Resume Evaluation System

This notebook demonstrates a LangGraph-based system for evaluating resumes and generating appropriate responses.

## Setup Requirements
1. Create a `.env` file in the project directory
2. Add your Google API key: `GOOGLE_API_KEY=your_api_key_here`
3. The `.env` file is included in `.gitignore` to keep your API key secure

In [40]:
%pip install -q langgraph langchain langchain_google_genai python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [41]:
%pip install langchain_google_genai

import os
from dotenv import load_dotenv
from typing import TypedDict, Optional
from langgraph.graph import StateGraph
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.schema import HumanMessage

# Load environment variables from .env file
load_dotenv()

Note: you may need to restart the kernel to use updated packages.


True

In [42]:
# Verify that the API key is loaded
api_key = os.getenv("GOOGLE_API_KEY")
if api_key:
    print("✅ API key loaded successfully")
    print(f"API key starts with: {api_key[:10]}...")
else:
    print("❌ API key not found. Please check your .env file")
    print("Make sure you have a .env file with GOOGLE_API_KEY=your_api_key")

✅ API key loaded successfully
API key starts with: AIzaSyCtn2...


In [31]:
class GraphsState(TypedDict):
    resume : Optional[str]
    qualified : Optional[bool]
    message : Optional[str]

In [43]:
llm = ChatGoogleGenerativeAI(
    model = "gemini-1.5-flash",
    temperature = 0.1,
    api_key = os.getenv("GOOGLE_API_KEY")
)

In [33]:
def load_resume(state):
    resume = '''Name : John Doe
    skills : Python, Data Analysis, Machine Learning
    experience : 5 years in software development
    education : B.E in Computer Science
    certifications : Certified Data Scientist
    '''
    return {"resume" : resume}

In [34]:
def check_qualification(state):
    resume = state["resume"]
    prompt = f""" Based on the following resume, determine if the candidate is qualified for a data scientist position with at least 1 year of experience in data science and machine learning.

    Answer only with "Qualified" or "Not Qualified".\n\n{resume}

    """
    response = llm.invoke([HumanMessage(content=prompt)])
    if "qualified" in response.content.lower():
        return {"qualified" : True, "message" : response.content}
    else:
        return {"qualified" : False, "message" : response.content}

In [35]:
def write_positive(state):
    resume = state["resume"]
    prompt = f''' Write a positive message to the candidate based on the following resume,
    encouraging them to apply for the data scientist position.\n\n{resume}
    '''
    response = llm.invoke([HumanMessage(content=prompt)])
    return {"message" : response.content}

In [36]:
def write_negative(state):
    resume = state["resume"]
    prompt = f"""Write a polite rejection message to the candidate based on the following resume,
    explaining why they are not qualified for the data scientist position.
    \n\n {resume}
    """
    response = llm.invoke([HumanMessage(content=prompt)])
    return {"message": response.content}

In [37]:
def route_decision(state):
    return "positive" if state["qualified"]  else "negative"

In [38]:
builder = StateGraph(GraphsState)
builder.add_node("load_resume",load_resume)
builder.add_node("check_qualification",check_qualification)
builder.add_node("positive",write_positive)
builder.add_node("negative",write_negative)

# Set up the graph structure
builder.set_entry_point("load_resume")
builder.add_edge("load_resume", "check_qualification")
builder.add_conditional_edges("check_qualification", route_decision)
builder.add_edge("positive", "__end__")
builder.add_edge("negative", "__end__")

In [39]:
graph = builder.compile()
output = graph.invoke({})



print("Resume")
print(output["resume"])
print("\n Result:","Qualified " if output["qualified"] else "Not Qualified")
print("\n Final Message: ")
print(output["message"])

Resume
Name : John Doe
    skills : Python, Data Analysis, Machine Learning
    experience : 5 years in software development
    education : B.E in Computer Science
    certifications : Certified Data Scientist
    

 Result: Qualified 

 Final Message: 
Subject: Data Scientist Opportunity - A Great Fit for Your Skills!

Dear John,

Your resume caught our eye, particularly your combination of strong software development experience and your Certified Data Scientist credential.  Five years in software development provides a solid foundation, and your skills in Python, data analysis, and machine learning directly align with the requirements of our open Data Scientist position.

We're looking for someone with a proven ability to leverage data to drive impactful results, and your background suggests you'd be a valuable asset to our team.  We encourage you to apply and share more about your experience and how your skills can contribute to [Company Name]'s success.

You can find the full job 